In [1]:
import pyodbc
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Configuración de la cadena de conexión
server = '192.168.5.23'  # Dirección del servidor
database = 'Sic_Grp'  # Nombre de la base de datos
username = 'UsrOptEstiba'  # Usuario proporcionado
password = '-20241204-'  # Contraseña proporcionada
driver = '{ODBC Driver 17 for SQL Server}'  # Driver ODBC

In [3]:
# Crear la conexión
try:
    connection = pyodbc.connect(
        f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}"
    )
    print("Conexión exitosa a la base de datos.")

    # Crear un cursor para ejecutar consultas
    cursor = connection.cursor()

    # Consulta para obtener datos de la vista filtrando por almacén 'bcn'
    query = """
    SELECT * 
    FROM sic_grp.dbo.TXALPartidaEstanciaOptEstibaV 
    WHERE almacenalias='bcn'
    """
    
    # Ejecutar la consulta
    cursor.execute(query)
    
    # Get field (column) names
    field_names = [column[0] for column in cursor.description]

    # Obtener los resultados
    results = cursor.fetchall()
    
except Exception as e:
    print(f"Error conectando a la base de datos: {e}")
    
finally:
    if 'connection' in locals() and connection:
        connection.close()
        print("Conexión cerrada.")

Conexión exitosa a la base de datos.
Conexión cerrada.


In [4]:
field_names = [name.strip() for name in field_names]
results = [tuple(row) for row in results] 
assert np.shape(results)[1] == len(field_names)

In [5]:
# Convert results to a DataFrame
results_df = pd.DataFrame(results, columns=field_names)

In [6]:
results_df['Remontable'] = results_df['NoRemontable'].map({0: 1, 1:0})
results_df['CodigoViaje'] = 0
results_df['FechaCargaContenedor'] = 0
results_df['Volumen'] = results_df['PartidaLargo']*results_df['PartidaAncho']*results_df['PartidaAlto']

In [7]:
mapper = {'ExpedicionCodigo': 'Expedicion',
        'PartidaKg': 'PesoKg', 
        'PartidaLargo': 'LargoCm', 
        'PartidaAncho': 'AnchoCm', 
        'PartidaAlto': 'AltoCm',
        'PartidaCodigoPartida': 'Partida',
        'PartidaFechaEntrada': 'FechaEntradaAlmacen',
        'PartidaTipo': 'TipoPartida'}

results_df = results_df.rename(columns=mapper)
results_df.columns

Index(['AlmacenCodigo', 'AlmacenAlias', 'ExpedicionDestino', 'Expedicion',
       'ExpedicionAdr', 'ExpedicionLevanteExportDt', 'ExpedicionCliente',
       'FechaEntradaAlmacen', 'Partida', 'PesoKg', 'LargoCm', 'AltoCm',
       'AnchoCm', 'PasadaPorVolumetrica', 'DatosVolumetricosClonados',
       'TipoPartida', 'NoRemontable', 'Remontable', 'CodigoViaje',
       'FechaCargaContenedor', 'Volumen'],
      dtype='object')

In [8]:
input_df = results_df[(results_df['ExpedicionDestino'] == 'PMI') & (results_df['PasadaPorVolumetrica'] == 1) & (results_df['TipoPartida'] == 'Palet')]
input_df.to_excel('input_RCH/test_1.xlsx')
len(input_df)

8